In [1]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Load the dictionary from the pickle file
with open('mydict_dict.pickle', 'rb') as file:
    mydict = pickle.load(file)

In [3]:
mustra = '''
<!DOCTYPE html>
<html lang="sk">
<head>
<meta charset="UTF-8">
<title>Liturgický kalendár 2023, skúšobná verzia</title>
<style>
    table {
        width: 75%; 
        margin-left: auto; 
        margin-right: auto;
        font-family: Arial, Helvetica, sans-serif; 
        line-height: 1.5;
        border: 1px solid gray; 
        border-collapse: collapse; 
        padding: 1rem;    
        margin-bottom: 100px;              
    }

    td {
        white-space: pre-wrap;
        border: 1px solid gray;
        border-collapse: collapse;
        padding: 1rem;        
    }

    th {
        font-size: 1.1rem;    
        border: 1px solid gray;
        border-collapse: collapse;
        padding: 1rem;          
    }
    
    .meniny {
        text-wrap: nowrap;
        text-align: center
    }

    tbody {
        border-style: ridge;
        border-width: 5px;
        border-color: coral
    }

    tr td:last-child {
        /* text-align: center; */
        font-size: 0.75rem;
    }      
</style>
</head>
<body>
<table>
<thead>
<th>deň</th>
<th>v liturgii</th>
<th>poznámka</th>
</thead>
</table>
</body>
</html>
'''

In [4]:
soup = BeautifulSoup(mustra , 'lxml')

In [5]:
for key in mydict.keys():

    # týždne sú oddelené hrubšími čiarami, jednotlivé týždne sú zabalené v <tbody> elementoch
    current_date = pd.to_datetime(key)
    if (key == '2023-01-01') or current_date.day_name('sk_SK')=='Pondelok':
        tbody_here = soup.new_tag('tbody') # create new tbody here
        soup.table.append(tbody_here)

    tr = soup.new_tag('tr')
    # tr['id'] = key # id='2023-01-01'

    velkost = len(mydict[key]) # length of nested dictionary

    for variable in mydict[key].keys(): # prechádzame nested dictionary
        if variable == 'meniny': # pole "meniny"
            td2 = soup.new_tag('td')
            td2['id'] = key # id='2023-01-01'
            td2['class']='meniny' # na túto class je napojené CSS, aby sa nezalamoval m_string.

            # meniny string obohatený o ďalšie údaje o dni, this will be crucial
            meniny = mydict[key][variable]
            m_string = str(current_date.day)+'. '+current_date.month_name('sk_SK').lower()+'\n'+current_date.day_name('sk_SK')+'\n<i>'+meniny+'</i>'
            app = BeautifulSoup(m_string,'html.parser')
            td2.append(app)

            if velkost>2:
                td2['rowspan']=str(velkost-1)      

            tr.append(td2)

        else: # zvyšné polia (0,1,2)
            td2 = soup.new_tag('td')
            td2.string = mydict[key][variable][0] # v liturgii... nultý (prvý) údaj v liste
            tr.append(td2)

            td2 = soup.new_tag('td')
            td2.string = mydict[key][variable][1] # poznámka... prvý (druhý) údaj v liste
            tr.append(td2)

            if velkost>2: # ak je v dni je viac možností, musíme pridať ďalší riadok (row)
                tbody_here.append(tr)
                tr = soup.new_tag('tr')

    tbody_here.append(tr)

In [6]:
# soup

In [7]:
with open('onepage_plain.html', mode='w', encoding='utf8') as f:
    f.write(str(soup))

In [8]:
# done.